# TensorFlow上的Fizz Buzz

原文（Source）：[Joel Grus – Fizz Buzz in Tensorflow](http://joelgrus.com/2016/05/23/fizz-buzz-in-tensorflow/)  
原作者（Original Author）：[Joel Grus](http://github.com/joelgrus)  
翻譯（Translator）：[Elvis YJ. Lin](https://github.com/elvisyjlin)

## 正文

**面試官：**歡迎你來！要不要喝杯咖啡或甚麼呢？需要休息一下嗎？  
**我：**不了，我大概已經喝夠多了。  
**面試官：**那好。你能夠在白板上寫程式嗎？  
**我：**我通常都是那樣寫的！  
**面試官：**...  
**我：**開個玩笑而已啦。  
**面試官：**好，所以你熟悉「Fizz Buzz」問題嗎？  
**我：**...  
**面試官：**額…這是肯定還是否定的意思？  
**我：**...我不敢相信你會問我這種問題。  
**面試官：**很好，我要你印出數字1到100，但是有例外，遇到能被3整除的數字時要印「fizz」，能被5整除的數字要印「buzz」，而能被15整除的數字則要印出「fizzbuzz」。  
**我：**我對這個很熟。  
**面試官：**那太好了，根據我們的經驗，不會這問題的人不適合在這兒工作。  
**我：**...  
**面試官：**這裡有白板筆和板擦可以用。  
**我：**（思考幾分鐘後...）  
**面試官：**需要我幫忙起個頭嗎？  
**我：**不用不用，我要開始了。讓我們先導入一些標準代碼吧。

In [1]:
import numpy as np
import tensorflow as tf

**面試官：**嗯...你瞭解這是*fizzbuzz*問題吧？  
**我：**當然！我們來看一下模型的部分。我打算簡單寫，使用一個只帶一個隱藏層的多層感知器。  
**面試官：**感知器？  
**我：**也叫做神經網路，或是你想怎麼稱呼它都可以。我們希望輸入是數字，而輸出是正確的「fizzbuzz」表示法。準確來說，我們必須把每個輸入活化成向量。一個簡單的方法是將它們轉為二進制。  
**面試官：**二進制？  
**我：**是啊，你也懂的，0和1嘛！像這樣：

In [2]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

**面試官：**（瞪著白板許久...）  
**我：**而我們的輸出會是一個fizzbuzz的獨熱編碼，它的第一個位置表示「數字本身」，第二個位置則是「fizz」，以此類推（第三個是「buzz」，第四個是「fizzbuzz」）：

In [3]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else            : return np.array([1, 0, 0, 0])

**面試官：**很好，我想這樣已經很足夠了！  
**我：**這才剛設置好而已呢！現在我們需要來產生些訓練資料。訓練資料使用1到100間的數字是作弊的，所以我們來讓它用到1024之前其餘的數字上面訓練吧！

In [4]:
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])

**面試官：**...  
**我：**再來需要在TensorFlow中建立我們的模型。我想想，我不太確定該使用多少個隱藏單元…大概10個吧？  
**面試官：**...  
**我：**也對，也許100個比較合適。沒關係我們之後還可以改它。

In [5]:
NUM_HIDDEN = 100

**我：**我們還需要一個寬度為NUM_DIGITS的輸入變數，與一個寬度為4的輸出變數。

In [6]:
X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

**面試官：**你打算繼續搞到什麼程度？  
**我：**不多，就兩層而已－－我不過用了一個隱藏層和一個輸出層。讓我們隨機地初始化神經元的權重吧：

In [7]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 4])

**我：**現在，我們已經準備好來定義模型了。如同我先前說過的，一層隱藏層，然後用…嗯…ReLU活化函數好了：

In [8]:
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

**我：**再來可以用歸一化指數交叉熵作為損失函數，並試著最小化它：

In [9]:
py_x = model(X, w_h, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

**面試官：**…  
**我：**接著，當然啦，預測結果就是輸出中，最大的那個類別。

In [10]:
predict_op = tf.argmax(py_x, 1)

**面試官：**在你做過頭前，我必須提醒一下，你應該解決的問題是產生1到100間的fizz buzz。  
**我：**哦！你提到重點了！這個`predict_op`函數將會輸出一個介於0到3的數字，但是我們想要的是「fizz buzz」輸出：

In [11]:
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

**面試官：**…  
**我：**於是呢，可以開始訓練模型囉！拿一個TensorFlow的Session，初始化裏面的變數：

In [12]:
# with tf.Session() as sess:
#     tf.global_variables_initializer().run()

**我：**現在，開始讓它跑吧！用1000次epoch來訓練如何？  
**面試官：**…  
**我：**好吧，也許有點不夠－－用10000次比較保險。而我們的訓練資料是按照順序排好的，我覺得不行，來讓它們每個epoch都隨機排序好了。
```
    for epoch in range(10000):
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]
```

In [13]:
NUM_EPOCHS = 10000

**我：**而每個epoch都讓模型批次訓練，嗯…我想想…每128為一組？

In [14]:
BATCH_SIZE = 128

**我：**於是訓練過程看起來像這樣
```
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
```

**我：**此外，我們可以印出訓練資料上的準確度，應該說為何不印呢？
```
        print(epoch, np.mean(np.argmax(trY, axis=1) == 
                             sess.run(predict_op, feed_dict={X: trX, Y: trY})))
```

**面試官：**你是認真的嗎...？  
**我：**是的，我認為這對觀察訓練準確度的成長很有幫助。  
**面試官：**…

In [15]:
sess = tf.Session()

with sess.as_default():
    tf.global_variables_initializer().run()
    
    for epoch in range(NUM_EPOCHS):
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]
        
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
        
        accuracy = np.mean(np.argmax(trY, axis=1) == 
                             sess.run(predict_op, feed_dict={X: trX, Y: trY}))
        print(epoch, accuracy)

0 0.534127843987
1 0.534127843987
2 0.534127843987
3 0.534127843987
4 0.534127843987
5 0.534127843987
6 0.534127843987
7 0.534127843987
8 0.534127843987
9 0.534127843987
10 0.534127843987
11 0.534127843987
12 0.534127843987
13 0.534127843987
14 0.534127843987
15 0.534127843987
16 0.534127843987
17 0.534127843987
18 0.534127843987
19 0.534127843987
20 0.534127843987
21 0.534127843987
22 0.534127843987
23 0.534127843987
24 0.534127843987
25 0.534127843987
26 0.534127843987
27 0.534127843987
28 0.534127843987
29 0.534127843987
30 0.534127843987
31 0.534127843987
32 0.534127843987
33 0.534127843987
34 0.534127843987
35 0.534127843987
36 0.534127843987
37 0.534127843987
38 0.534127843987
39 0.534127843987
40 0.534127843987
41 0.534127843987
42 0.534127843987
43 0.534127843987
44 0.534127843987
45 0.534127843987
46 0.534127843987
47 0.534127843987
48 0.534127843987
49 0.534127843987
50 0.534127843987
51 0.534127843987
52 0.534127843987
53 0.534127843987
54 0.534127843987
55 0.534127843987
56

440 0.534127843987
441 0.534127843987
442 0.534127843987
443 0.534127843987
444 0.534127843987
445 0.534127843987
446 0.534127843987
447 0.534127843987
448 0.534127843987
449 0.534127843987
450 0.534127843987
451 0.534127843987
452 0.534127843987
453 0.534127843987
454 0.534127843987
455 0.534127843987
456 0.534127843987
457 0.534127843987
458 0.534127843987
459 0.534127843987
460 0.534127843987
461 0.534127843987
462 0.534127843987
463 0.534127843987
464 0.534127843987
465 0.534127843987
466 0.534127843987
467 0.534127843987
468 0.534127843987
469 0.534127843987
470 0.534127843987
471 0.534127843987
472 0.534127843987
473 0.534127843987
474 0.534127843987
475 0.534127843987
476 0.534127843987
477 0.534127843987
478 0.534127843987
479 0.534127843987
480 0.534127843987
481 0.534127843987
482 0.534127843987
483 0.534127843987
484 0.534127843987
485 0.534127843987
486 0.534127843987
487 0.534127843987
488 0.534127843987
489 0.534127843987
490 0.534127843987
491 0.534127843987
492 0.534127

876 0.6511375948
877 0.628385698808
878 0.65872156013
879 0.655471289274
880 0.653304442037
881 0.650054171181
882 0.648970747562
883 0.63488624052
884 0.653304442037
885 0.6511375948
886 0.638136511376
887 0.652221018418
888 0.648970747562
889 0.638136511376
890 0.645720476706
891 0.637053087757
892 0.652221018418
893 0.660888407367
894 0.654387865655
895 0.640303358613
896 0.654387865655
897 0.654387865655
898 0.639219934995
899 0.653304442037
900 0.661971830986
901 0.647887323944
902 0.648970747562
903 0.656554712893
904 0.654387865655
905 0.655471289274
906 0.643553629469
907 0.62730227519
908 0.653304442037
909 0.638136511376
910 0.65872156013
911 0.655471289274
912 0.65872156013
913 0.660888407367
914 0.646803900325
915 0.659804983749
916 0.656554712893
917 0.640303358613
918 0.663055254605
919 0.667388949079
920 0.657638136511
921 0.641386782232
922 0.64247020585
923 0.661971830986
924 0.656554712893
925 0.661971830986
926 0.660888407367
927 0.65872156013
928 0.644637053088
929 

1296 0.776814734561
1297 0.784398699892
1298 0.775731310943
1299 0.784398699892
1300 0.791982665222
1301 0.78548212351
1302 0.796316359697
1303 0.786565547129
1304 0.783315276273
1305 0.797399783315
1306 0.788732394366
1307 0.796316359697
1308 0.78548212351
1309 0.784398699892
1310 0.807150595883
1311 0.795232936078
1312 0.791982665222
1313 0.798483206934
1314 0.776814734561
1315 0.795232936078
1316 0.778981581798
1317 0.791982665222
1318 0.778981581798
1319 0.800650054171
1320 0.788732394366
1321 0.783315276273
1322 0.794149512459
1323 0.791982665222
1324 0.783315276273
1325 0.778981581798
1326 0.78548212351
1327 0.759479956663
1328 0.784398699892
1329 0.800650054171
1330 0.795232936078
1331 0.733477789816
1332 0.773564463705
1333 0.787648970748
1334 0.795232936078
1335 0.798483206934
1336 0.790899241603
1337 0.778981581798
1338 0.774647887324
1339 0.791982665222
1340 0.793066088841
1341 0.800650054171
1342 0.796316359697
1343 0.810400866739
1344 0.776814734561
1345 0.789815817985
134

1708 0.875406283857
1709 0.837486457205
1710 0.876489707476
1711 0.865655471289
1712 0.842903575298
1713 0.88082340195
1714 0.878656554713
1715 0.87323943662
1716 0.829902491874
1717 0.835319609967
1718 0.840736728061
1719 0.886240520043
1720 0.881906825569
1721 0.872156013001
1722 0.875406283857
1723 0.884073672806
1724 0.887323943662
1725 0.823401950163
1726 0.868905742145
1727 0.878656554713
1728 0.881906825569
1729 0.860238353196
1730 0.838569880823
1731 0.88082340195
1732 0.869989165764
1733 0.872156013001
1734 0.834236186349
1735 0.876489707476
1736 0.889490790899
1737 0.859154929577
1738 0.847237269772
1739 0.877573131094
1740 0.875406283857
1741 0.889490790899
1742 0.867822318527
1743 0.881906825569
1744 0.875406283857
1745 0.888407367281
1746 0.881906825569
1747 0.8255687974
1748 0.899241603467
1749 0.875406283857
1750 0.881906825569
1751 0.876489707476
1752 0.867822318527
1753 0.838569880823
1754 0.893824485374
1755 0.886240520043
1756 0.87323943662
1757 0.882990249187
1758 0

...

6449 0.996749729144
6450 0.997833152763
6451 0.996749729144
6452 0.997833152763
6453 0.996749729144
6454 0.997833152763
6455 0.997833152763
6456 0.997833152763
6457 0.997833152763
6458 0.998916576381
6459 0.997833152763
6460 0.995666305525
6461 0.998916576381
6462 0.997833152763
6463 0.998916576381
6464 0.996749729144
6465 0.998916576381
6466 0.994582881907
6467 0.997833152763
6468 0.997833152763
6469 0.998916576381
6470 0.996749729144
6471 0.996749729144
6472 0.993499458288
6473 0.998916576381
6474 0.998916576381
6475 0.997833152763
6476 0.997833152763
6477 0.997833152763
6478 0.998916576381
6479 0.998916576381
6480 0.996749729144
6481 0.997833152763
6482 0.996749729144
6483 0.998916576381
6484 1.0
6485 0.997833152763
6486 0.997833152763
6487 0.997833152763
6488 0.997833152763
6489 0.997833152763
6490 0.998916576381
6491 0.997833152763
6492 0.997833152763
6493 0.998916576381
6494 0.997833152763
6495 0.997833152763
6496 0.998916576381
6497 0.996749729144
6498 0.997833152763
6499 0.9978

7003 1.0
7004 0.997833152763
7005 1.0
7006 1.0
7007 1.0
7008 1.0
7009 1.0
7010 1.0
7011 1.0
7012 1.0
7013 1.0
7014 1.0
7015 1.0
7016 0.997833152763
7017 0.997833152763
7018 0.998916576381
7019 1.0
7020 1.0
7021 0.997833152763
7022 1.0
7023 1.0
7024 1.0
7025 1.0
7026 1.0
7027 1.0
7028 0.998916576381
7029 1.0
7030 1.0
7031 1.0
7032 0.997833152763
7033 1.0
7034 1.0
7035 0.998916576381
7036 0.998916576381
7037 1.0
7038 1.0
7039 1.0
7040 0.997833152763
7041 1.0
7042 1.0
7043 1.0
7044 1.0
7045 1.0
7046 1.0
7047 0.998916576381
7048 1.0
7049 1.0
7050 1.0
7051 1.0
7052 1.0
7053 0.998916576381
7054 1.0
7055 1.0
7056 1.0
7057 1.0
7058 1.0
7059 1.0
7060 1.0
7061 0.998916576381
7062 1.0
7063 1.0
7064 1.0
7065 1.0
7066 1.0
7067 1.0
7068 1.0
7069 1.0
7070 1.0
7071 0.998916576381
7072 0.998916576381
7073 1.0
7074 1.0
7075 0.997833152763
7076 1.0
7077 1.0
7078 1.0
7079 1.0
7080 1.0
7081 0.998916576381
7082 0.997833152763
7083 0.997833152763
7084 1.0
7085 1.0
7086 1.0
7087 1.0
7088 1.0
7089 1.0
7090 1.0

7849 1.0
7850 1.0
7851 1.0
7852 1.0
7853 1.0
7854 1.0
7855 1.0
7856 1.0
7857 1.0
7858 1.0
7859 1.0
7860 1.0
7861 1.0
7862 1.0
7863 1.0
7864 1.0
7865 1.0
7866 1.0
7867 1.0
7868 1.0
7869 1.0
7870 1.0
7871 1.0
7872 1.0
7873 1.0
7874 1.0
7875 1.0
7876 1.0
7877 1.0
7878 1.0
7879 1.0
7880 1.0
7881 1.0
7882 1.0
7883 1.0
7884 0.997833152763
7885 1.0
7886 1.0
7887 1.0
7888 1.0
7889 1.0
7890 1.0
7891 1.0
7892 1.0
7893 1.0
7894 1.0
7895 1.0
7896 1.0
7897 1.0
7898 1.0
7899 1.0
7900 1.0
7901 1.0
7902 1.0
7903 1.0
7904 1.0
7905 1.0
7906 1.0
7907 1.0
7908 1.0
7909 1.0
7910 1.0
7911 1.0
7912 1.0
7913 1.0
7914 1.0
7915 1.0
7916 1.0
7917 1.0
7918 0.998916576381
7919 1.0
7920 1.0
7921 1.0
7922 1.0
7923 1.0
7924 1.0
7925 1.0
7926 1.0
7927 1.0
7928 1.0
7929 1.0
7930 1.0
7931 1.0
7932 1.0
7933 1.0
7934 1.0
7935 1.0
7936 1.0
7937 0.998916576381
7938 1.0
7939 1.0
7940 1.0
7941 0.998916576381
7942 1.0
7943 1.0
7944 1.0
7945 1.0
7946 1.0
7947 1.0
7948 1.0
7949 1.0
7950 1.0
7951 1.0
7952 1.0
7953 1.0
7954 1.0
79

8769 1.0
8770 1.0
8771 1.0
8772 1.0
8773 1.0
8774 1.0
8775 1.0
8776 1.0
8777 1.0
8778 1.0
8779 1.0
8780 1.0
8781 1.0
8782 1.0
8783 1.0
8784 1.0
8785 1.0
8786 1.0
8787 1.0
8788 1.0
8789 1.0
8790 1.0
8791 1.0
8792 1.0
8793 1.0
8794 1.0
8795 1.0
8796 1.0
8797 1.0
8798 1.0
8799 1.0
8800 1.0
8801 1.0
8802 1.0
8803 1.0
8804 1.0
8805 1.0
8806 1.0
8807 1.0
8808 1.0
8809 1.0
8810 1.0
8811 1.0
8812 1.0
8813 1.0
8814 1.0
8815 1.0
8816 1.0
8817 1.0
8818 1.0
8819 1.0
8820 1.0
8821 1.0
8822 1.0
8823 1.0
8824 1.0
8825 1.0
8826 1.0
8827 1.0
8828 1.0
8829 1.0
8830 1.0
8831 1.0
8832 1.0
8833 1.0
8834 1.0
8835 1.0
8836 1.0
8837 1.0
8838 1.0
8839 1.0
8840 1.0
8841 1.0
8842 1.0
8843 1.0
8844 1.0
8845 1.0
8846 1.0
8847 1.0
8848 1.0
8849 1.0
8850 1.0
8851 1.0
8852 1.0
8853 1.0
8854 1.0
8855 1.0
8856 1.0
8857 1.0
8858 1.0
8859 1.0
8860 1.0
8861 1.0
8862 1.0
8863 1.0
8864 1.0
8865 1.0
8866 1.0
8867 1.0
8868 1.0
8869 1.0
8870 1.0
8871 1.0
8872 1.0
8873 1.0
8874 1.0
8875 1.0
8876 1.0
8877 1.0
8878 1.0
8879 1.0
8

9694 1.0
9695 1.0
9696 1.0
9697 1.0
9698 1.0
9699 1.0
9700 1.0
9701 1.0
9702 1.0
9703 1.0
9704 1.0
9705 1.0
9706 1.0
9707 1.0
9708 1.0
9709 1.0
9710 1.0
9711 1.0
9712 1.0
9713 1.0
9714 1.0
9715 1.0
9716 1.0
9717 1.0
9718 1.0
9719 1.0
9720 1.0
9721 1.0
9722 1.0
9723 1.0
9724 1.0
9725 1.0
9726 1.0
9727 1.0
9728 1.0
9729 1.0
9730 1.0
9731 1.0
9732 1.0
9733 1.0
9734 1.0
9735 1.0
9736 1.0
9737 1.0
9738 1.0
9739 1.0
9740 1.0
9741 1.0
9742 1.0
9743 1.0
9744 1.0
9745 1.0
9746 1.0
9747 1.0
9748 1.0
9749 1.0
9750 1.0
9751 1.0
9752 1.0
9753 1.0
9754 1.0
9755 1.0
9756 1.0
9757 1.0
9758 1.0
9759 1.0
9760 1.0
9761 1.0
9762 1.0
9763 1.0
9764 1.0
9765 1.0
9766 1.0
9767 1.0
9768 1.0
9769 1.0
9770 1.0
9771 1.0
9772 1.0
9773 1.0
9774 1.0
9775 1.0
9776 1.0
9777 1.0
9778 1.0
9779 1.0
9780 1.0
9781 1.0
9782 1.0
9783 1.0
9784 1.0
9785 1.0
9786 1.0
9787 1.0
9788 1.0
9789 1.0
9790 1.0
9791 1.0
9792 1.0
9793 1.0
9794 1.0
9795 1.0
9796 1.0
9797 1.0
9798 1.0
9799 1.0
9800 1.0
9801 1.0
9802 1.0
9803 1.0
9804 1.0
9

**我：**好了，一旦模型訓練完成，就是Fizz Buzz的時間啦！我們的輸入應該要是1到100的二進制表示：

In [16]:
numbers = np.arange(1, 101)
teX = np.transpose(binary_encode(numbers, NUM_DIGITS))

**我：**而輸出則是將模型輸出丟進`fizz_buzz`函數後的結果：

In [17]:
teY = sess.run(predict_op, feed_dict={X: teX})
output = np.vectorize(fizz_buzz)(numbers, teY)

print(output)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' 'fizz' '22' '23' 'fizz' '25'
 '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz' 'fizz' '37'
 '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' '45' '46' '47' 'fizz' '49' 'buzz'
 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59' 'fizzbuzz' '61' '62'
 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz' '71' 'fizz' '73' '74'
 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' 'fizz' '82' '83' 'fizz' 'buzz'
 '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94' 'buzz' 'fizz' '97'
 '98' 'fizz' 'buzz']


**面試官：**…  
**我：**這個應該就是你要的Fizz Buzz了！  
**面試官：**真的，已經夠了。謝謝你，我們保持連絡。  
**我：**保持聯絡，聽起來真有希望。  
**面試官：**…

## 後記

後來我沒得到這個職位。於是我實際上嘗試執行這段程式碼（code on Github），結果竟然有些輸出是錯誤的！機器學習，真是謝謝你了！

In [18]:
actuals = [fizz_buzz(i, fizz_buzz_encode(i).argmax()) for i in numbers]

for i, (predicted, actual) in enumerate(zip(output, actuals)):
    if predicted != actual:
        print("{0} {1} {2}".format(i+1, predicted, actual))

25 25 buzz
45 45 fizzbuzz


我想，當初應該用深一點的網路來做的。

## 備註

許多字詞為專有名詞，不同地方有不同翻法，以下列出我自己的翻譯對照：

**中文** | **English**
--- | ---
活化 | activation
活化函數 | activation function
批次 | batch
隱藏單元 | hidden unit
隱藏層 | hidden layer
輸入變數 | input variable
損失函數 | loss function
機器學習 | machine learning
模型 | model
多層感知器 | multi-layer-perceptron
神經網路 | neural network
獨熱編碼 | one-hot encoding
輸出層 | output layer
輸出變數 | output variable
感知器 | perceptron
歸一化指數交叉熵 | softmax cross-entropy
導入標準代碼 | standard imports
訓練資料 | training data